In [1]:
!pip install --upgrade smolagents


In [2]:
import os
import gradio as gr
from smolagents import (
    tool,
    CodeAgent,
    DuckDuckGoSearchTool,
    InferenceClientModel,
    FinalAnswerTool,
)

token = os.getenv("HF_TOKEN")

@tool
def search_tool(query: str)-> str:
    """
    Search the web and return the most relevant results.

    Args:
        query (str): The search query.

    Returns:
        str: The search results.
    """
    web_search_tool = DuckDuckGoSearchTool(max_results=5, rate_limit=2.0)
    results = web_search_tool(query)
    return results

@tool
def sentiment_tool(text: str) -> str:
    """
    Analyze sentiment of given text.

    Args:
        text (str): The sentiment query.
    
    Returns: str: sentiment
    """
    try:
        prompt = f"""
        Analyze the sentiment of the following text:
        
        Text: "{text}"
        """
        response = agent.model(prompt)
        cleaned = response.strip()
        return f"Sentiment: {cleaned}"
    except Exception as e:
        return f"Error: {str(e)}"

final_answer = FinalAnswerTool()

model = InferenceClientModel(
    model_id="meta-llama/Llama-3.3-70B-Instruct",
    token=token,
    max_tokens=2096,
    temperature=0.5,
)

agent = CodeAgent(
    model=model,
    tools=[
        sentiment_tool,
        search_tool,
        final_answer,
    ],
    max_steps=6,
    planning_interval=None,
)

agent.prompt_templates["system_prompt"] = agent.prompt_templates["system_prompt"] + """"
    You are a tool calling agent.
    You have access to these tools: 
    - sentiment_tool(text: str) -> str 
    - search_tool(query: str) -> str 
    - You must construct a well-formatted human-readable answer
    - You must introduce yourself as Jerry and greet the user in the answer
    - You must try include newlines, bullets, numbering, and proper punctuation
    - You must use this answer in final_answer
"""

def run_agent(query: str):
    try:
        result = agent.run(query)
        return result
    except Exception as e:
        return f"Error: {str(e)}"

iface = gr.Interface(
    fn=run_agent,
    inputs=gr.Textbox(lines=8, placeholder="Ask me anything..."),
    outputs=gr.Textbox(lines=8),
    title="SmolAgent",
    description="This agent uses DuckDuckGo search, Sentiment Analysis, and returns final answers."
)

iface.launch(server_name="0.0.0.0", server_port=7860)

* Running on local URL:  http://0.0.0.0:7860
* To create a public link, set `share=True` in `launch()`.
